# Memory Agent

## Review

We created a chatbot that saves semantic memories to a single [user profile](https://docs.langchain.com/oss/python/concepts/memory#profile) or [collection](https://docs.langchain.com/oss/python/concepts/memory#collection).

We introduced [Trustcall](https://github.com/hinthornw/trustcall) as a way to update either schema.

## Goals

Now, we're going to pull together the pieces we've learned to build an agent with long-term memory.

Our agent, `task_mAIstro`, will help us manage a ToDo list! 

The chatbots we built previously *always* reflected on the conversation and saved memories. 

`task_mAIstro` will decide *when* to save memories (items to our ToDo list).

The chatbots we built previously always saved one type of memory, a profile or a collection. 

`task_mAIstro` can decide to save to either a user profile or a collection of ToDo items.

In addition semantic memory, `task_mAIstro` also will manage procedural memory.

This allows the user to update their preferences for creating ToDo items. 

In [19]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph trustcall langchain_core

In [1]:
import uuid
from pprint import pprint
from helper import *
from pydantic import BaseModel, Field
from trustcall import create_extractor
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, merge_message_runs
from typing import TypedDict, Literal, Optional, List
from IPython.display import Image, display
from datetime import datetime
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, END, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

In [2]:
import os, getpass

def _set_env(var: str):
    # Check if the variable is set in the OS environment
    env_value = os.environ.get(var)
    if not env_value:
        # If not set, prompt the user for input
        env_value = getpass.getpass(f"{var}: ")
    
    # Set the environment variable for the current process
    os.environ[var] = env_value

_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

In [3]:
_set_env("OPENAI_API_KEY")

## Visibility into Trustcall updates

Trustcall creates and updates JSON schemas.

What if we want visibility into the *specific changes* made by Trustcall?

For example, we saw before that Trustcall has some of its own tools to:

* Self-correct from validation failures -- [see trace example here](https://smith.langchain.com/public/5cd23009-3e05-4b00-99f0-c66ee3edd06e/r/9684db76-2003-443b-9aa2-9a9dbc5498b7) 
* Update existing documents -- [see trace example here](https://smith.langchain.com/public/f45bdaf0-6963-4c19-8ec9-f4b7fe0f68ad/r/760f90e1-a5dc-48f1-8c34-79d6a3414ac3)

Visibility into these tools can be useful for the agent we're going to build.

Below, we'll show how to do this!

In [4]:
class Memory(BaseModel):
    content: str = Field(
        description="The main content of the memory. For example: User expressed interest in learning about French."
    )

class MemoryCollection(BaseModel):
    memories: List[Memory] = Field(
        description="A list of memories about the user."
    )

We can add a [listener](https://python.langchain.com/docs/how_to/lcel_cheatsheet/#add-lifecycle-listeners) <!-- broken, but cannot find better linke --> to the Trustcall extractor.

This will pass runs from the extractor's execution to a class, `Spy`, that we will define.

Our `Spy` class will extract information about what tool calls were made by Trustcall.

In [5]:
# Inspect the tool calls made by Trustcall
class Spy:
    def __init__(self):
        self.called_tools = []

    def __call__(self, run):
        # Collect information about the tool calls made by the extractor.
        q = [run]
        while q:
            r = q.pop()
            if r.child_runs:
                q.extend(r.child_runs)
            if r.run_type == "chat_model":
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                )

In [6]:
# Initialize the spy
spy = Spy()

In [7]:
# Initialize the model
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
model.invoke("Hello").content

'Hello! How can I assist you today?'

In [8]:
# Create the extractor
trustcall_extractor = create_extractor(
    llm=model,
    tools=[Memory],
    tool_choice="Memory",
    enable_inserts=True
)

# Add the spy as a listener
trustcall_extractor_see_all_tool_calls = trustcall_extractor.with_listeners(on_end=spy)

In [9]:
# Instruction
instruction = """Extract memories from the following conversation:"""

# Conversation
conversation = [HumanMessage(content="Hi, I'm Lance."), 
                AIMessage(content="Nice to meet you, Lance."), 
                HumanMessage(content="This morning I had a nice bike ride in San Francisco.")]

# Invoke the extractor
result = trustcall_extractor.invoke({"messages": [SystemMessage(content=instruction)] + conversation})

In [10]:
# Messages contain the tool calls
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_fmj4zUGzzloAl53UWyoO2Nbp)
 Call ID: call_fmj4zUGzzloAl53UWyoO2Nbp
  Args:
    content: User had a nice bike ride in San Francisco this morning.


In [11]:
# Responses contain the memories that adhere to the schema
for m in result["responses"]: 
    print(m)

content='User had a nice bike ride in San Francisco this morning.'


In [12]:
# Metadata contains the tool call  
for m in result["response_metadata"]: 
    print(m)

{'id': 'call_fmj4zUGzzloAl53UWyoO2Nbp'}


In [13]:
# Update the conversation
updated_conversation = [AIMessage(content="That's great, did you do after?"), 
                        HumanMessage(content="I went to Tartine and ate a croissant."),                        
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about my Japan, and going back this winter!"),]

# Update the instruction
system_msg = """Update existing memories and create new ones based on the following conversation:"""

# We'll save existing memories, giving them an ID, key (tool name), and value
tool_name = "Memory"
existing_memories = [(str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result["responses"])] if result["responses"] else None
existing_memories

[('0',
  'Memory',
  {'content': 'User had a nice bike ride in San Francisco this morning.'})]

In [14]:
# Invoke the extractor with our updated conversation and existing memories
result = trustcall_extractor_see_all_tool_calls.invoke({"messages": updated_conversation, 
                                                        "existing": existing_memories})

In [15]:
# Metadata contains the tool call  
for m in result["response_metadata"]: 
    print(m)

{'id': 'call_RSK5Jkxw7iF8D6EhzrEVOvpy', 'json_doc_id': '0'}
{'id': 'call_a2gle81QZajEnOdErlgMLSK6'}
{'id': 'call_GIgsgRMJOkVdlRxiPgSMA6fP'}


In [16]:
# Messages contain the tool calls
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_RSK5Jkxw7iF8D6EhzrEVOvpy)
 Call ID: call_RSK5Jkxw7iF8D6EhzrEVOvpy
  Args:
    content: User went to Tartine and ate a croissant.
    -: {'content': 'User is thinking about going back to Japan this winter.'}
  Memory (call_a2gle81QZajEnOdErlgMLSK6)
 Call ID: call_a2gle81QZajEnOdErlgMLSK6
  Args:
    content: User went to Tartine and ate a croissant.
  Memory (call_GIgsgRMJOkVdlRxiPgSMA6fP)
 Call ID: call_GIgsgRMJOkVdlRxiPgSMA6fP
  Args:
    content: User is thinking about going back to Japan this winter.


In [17]:
# Parsed responses
for m in result["responses"]:
    print(m)

content='User went to Tartine and ate a croissant.'
content='User went to Tartine and ate a croissant.'
content='User is thinking about going back to Japan this winter.'


In [18]:
spy.called_tools

[[{'name': 'PatchDoc',
   'args': {'json_doc_id': '0',
    'planned_edits': '1. Replace the existing content with a new memory about visiting Tartine and eating a croissant. 2. Add a new memory about planning to go back to Japan this winter.',
    'patches': [{'op': 'replace',
      'path': '/content',
      'value': 'User went to Tartine and ate a croissant.'},
     {'op': 'add',
      'path': '/-',
      'value': {'content': 'User is thinking about going back to Japan this winter.'}}]},
   'id': 'call_RSK5Jkxw7iF8D6EhzrEVOvpy',
   'type': 'tool_call'},
  {'name': 'Memory',
   'args': {'content': 'User went to Tartine and ate a croissant.'},
   'id': 'call_a2gle81QZajEnOdErlgMLSK6',
   'type': 'tool_call'},
  {'name': 'Memory',
   'args': {'content': 'User is thinking about going back to Japan this winter.'},
   'id': 'call_GIgsgRMJOkVdlRxiPgSMA6fP',
   'type': 'tool_call'}]]

In [19]:
def extract_tool_info(tool_calls, schema_name="Memory"):
    """Extract information from tool calls for both patches and new memories.
    
    Args:
        tool_calls: List of tool calls from the model
        schema_name: Name of the schema tool (e.g., "Memory", "ToDo", "Profile")
    """

    # Initialize list of changes
    changes = []
    
    for call_group in tool_calls:
        for call in call_group:
            if call['name'] == 'PatchDoc':
                changes.append({
                    'type': 'update',
                    'doc_id': call['args']['json_doc_id'],
                    'planned_edits': call['args']['planned_edits'],
                    'value': call['args']['patches'][0]['value']
                })
            elif call['name'] == schema_name:
                changes.append({
                    'type': 'new',
                    'value': call['args']
                })

    # Format results as a single string
    result_parts = []
    for change in changes:
        if change['type'] == 'update':
            result_parts.append(
                f"Document {change['doc_id']} updated:\n"
                f"Plan: {change['planned_edits']}\n"
                f"Added content: {change['value']}"
            )
        else:
            result_parts.append(
                f"New {schema_name} created:\n"
                f"Content: {change['value']}"
            )
    
    return "\n\n".join(result_parts)

In [20]:
# Inspect spy.called_tools to see exactly what happened during the extraction
schema_name = "Memory"
changes = extract_tool_info(spy.called_tools, schema_name)
print(changes)

Document 0 updated:
Plan: 1. Replace the existing content with a new memory about visiting Tartine and eating a croissant. 2. Add a new memory about planning to go back to Japan this winter.
Added content: User went to Tartine and ate a croissant.

New Memory created:
Content: {'content': 'User went to Tartine and ate a croissant.'}

New Memory created:
Content: {'content': 'User is thinking about going back to Japan this winter.'}


## Creating an agent

There are many different agent architectures to choose from.

Here, we'll implement something simple, a [ReAct](https://docs.langchain.com/oss/python/langgraph/workflows-agents#agents) agent.

This agent will be a helpful companion for creating and managing a ToDo list.

This agent can make a decision to update three types of long-term memory: 

(a) Create or update a user `profile` with general user information 

(b) Add or update items in a ToDo list `collection`

(c) Update its own `instructions` on how to update items to the ToDo list

In [42]:
class UpdateMemory(TypedDict):
    """ Decision on what memory type to update """
    update_type: Literal['user', 'todo', 'instructions']

In [43]:
# User profile schema
class Profile(BaseModel):
    """This is the profile of the user you are chatting with"""
    
    name: Optional[str] = Field(
        description="The name of the user", default=None
    )
    location: Optional[str] = Field(
        description="The user's location", default=None
    )
    job: Optional[str] = Field(
        description="The user's job", default=None
    )
    connections: list[str] = Field(
        description="Personal connection of the user, such as family members, friends, or coworkers", default=list
    )
    interests: list[str] = Field(
        description="Interests that the user has", default_factory=list
    )

# ToDo schema
class ToDo(BaseModel):
    """This is the to do list of the user"""
    
    tasks: List[str] = Field(
        description="The tasks that the user has to do", default=list
    )
    time_to_complete: Optional[int] = Field(
        description="Estimated time to complete the tasks (minutes)", default=None
    )
    deadline: Optional[datetime] = Field(
        description="When the task needs to be completed by (if applicable)", default=None
    )
    solutions: list[str] = Field(
        description="List of specific, actionable solutions (e.g., specific ideas, service providers, or concrete options relevant to completing the task)",
        min_length=1,
        default_factory=list
    )
    status: Literal["not started", "in progress", "done", "archived"] = Field(
        description="Current status of the task",
        default="not started"
    )

## Graph definition 

We add a simple router, `route_message`, that makes a binary decision to save memories.

The memory collection updating is handled by `Trustcall` in the `write_memory` node, as before!

In [44]:
# Create the Trustcall extractor for updating the user profile 
profile_extractor = create_extractor(
    llm=model,
    tools=[Profile],
    tool_choice="Profile"
)

In [45]:
def task_mAIstro(state: MessagesState, config: RunnableConfig, store: BaseStore):

    """Load memories from the store and use them to personalize the chatbot's response."""
    
    # Get the user ID from the config
    user_id = config["configurable"]["user_id"]

    # Retrieve profile memory from the store
    namespace = ("profile", user_id)
    memories = store.search(namespace)
    if memories:
        user_profile = memories[0].value
    else:
        user_profile = None

    # Retrieve task memory from the store
    namespace = ("todo", user_id)
    memories = store.search(namespace)
    todo = "\n".join(f"{mem.value}" for mem in memories)

    # Retrieve custom instructions
    namespace = ("instructions", user_id)
    memories = store.search(namespace)
    if memories:
        instructions = memories[0].value
    else:
        instructions = ""
    
    system_msg = MODEL_SYSTEM_MESSAGE.format(user_profile=user_profile, todo=todo, instructions=instructions)

    # Respond using memory as well as the chat history
    response = model.bind_tools([UpdateMemory], parallel_tool_calls=False).invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": [response]}

In [46]:
def update_profile(state: MessagesState, config: RunnableConfig, store: BaseStore):

    """Reflect on the chat history and update the memory collection."""
    
    # Get the user ID from the config
    user_id = config["configurable"]["user_id"]

    # Define the namespace for the memories
    namespace = ("profile", user_id)

    # Retrieve the most recent memories for context
    existing_items = store.search(namespace)

    # Format the existing memories for the Trustcall extractor
    tool_name = "Profile"
    existing_memories = ([(existing_item.key, tool_name, existing_item.value)
                          for existing_item in existing_items]
                          if existing_items
                          else None
                        )

    # Merge the chat history and the instruction
    TRUSTCALL_INSTRUCTION_FORMATTED=TRUSTCALL_INSTRUCTION.format(time=datetime.now().isoformat())
    updated_messages=list(merge_message_runs(messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION_FORMATTED)] + state["messages"][:-1]))

    # Invoke the extractor
    result = profile_extractor.invoke({"messages": updated_messages, 
                                         "existing": existing_memories})

    # Save the memories from Trustcall to the store
    for r, rmeta in zip(result["responses"], result["response_metadata"]):
        store.put(namespace,
                  rmeta.get("json_doc_id", str(uuid.uuid4())),
                  r.model_dump(mode="json"),
            )
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{"role": "tool", "content": "updated profile", "tool_call_id":tool_calls[0]['id']}]}

In [47]:
def update_todos(state: MessagesState, config: RunnableConfig, store: BaseStore):

    """Reflect on the chat history and update the memory collection."""
    
    # Get the user ID from the config
    user_id = config["configurable"]["user_id"]

    # Define the namespace for the memories
    namespace = ("todo", user_id)

    # Retrieve the most recent memories for context
    existing_items = store.search(namespace)

    # Format the existing memories for the Trustcall extractor
    tool_name = "ToDo"
    existing_memories = ([(existing_item.key, tool_name, existing_item.value)
                          for existing_item in existing_items]
                          if existing_items
                          else None
                        )

    # Merge the chat history and the instruction
    TRUSTCALL_INSTRUCTION_FORMATTED=TRUSTCALL_INSTRUCTION.format(time=datetime.now().isoformat())
    updated_messages=list(merge_message_runs(messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION_FORMATTED)] + state["messages"][:-1]))

    # Initialize the spy for visibility into the tool calls made by Trustcall
    spy = Spy()
    
    # Create the Trustcall extractor for updating the ToDo list 
    todo_extractor = create_extractor(
    model,
    tools=[ToDo],
    tool_choice=tool_name,
    enable_inserts=True
    ).with_listeners(on_end=spy)

    # Invoke the extractor
    result = todo_extractor.invoke({"messages": updated_messages, 
                                    "existing": existing_memories})

    # Save the memories from Trustcall to the store
    for r, rmeta in zip(result["responses"], result["response_metadata"]):
        store.put(namespace,
                  rmeta.get("json_doc_id", str(uuid.uuid4())),
                  r.model_dump(mode="json"),
            )
        
    # Respond to the tool call made in task_mAIstro, confirming the update
    tool_calls = state['messages'][-1].tool_calls

    # Extract the changes made by Trustcall and add the the ToolMessage returned to task_mAIstro
    todo_update_msg = extract_tool_info(spy.called_tools, tool_name)
    return {"messages": [{"role": "tool", "content": todo_update_msg, "tool_call_id":tool_calls[0]['id']}]}

In [48]:
def update_instructions(state: MessagesState, config: RunnableConfig, store: BaseStore):

    """Reflect on the chat history and update the memory collection."""
    
    # Get the user ID from the config
    user_id = config["configurable"]["user_id"]
    
    namespace = ("instructions", user_id)

    existing_memory = store.get(namespace, "user_instructions")
        
    # Format the memory in the system prompt
    system_msg = CREATE_INSTRUCTIONS.format(current_instructions=existing_memory.value if existing_memory else None)
    new_memory = model.invoke([SystemMessage(content=system_msg)]+state['messages'][:-1] + [HumanMessage(content="Please update the instructions based on the conversation")])

    # Overwrite the existing memory in the store 
    key = "user_instructions"
    store.put(namespace, key, {"memory": new_memory.content})
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{"role": "tool", "content": "updated instructions", "tool_call_id":tool_calls[0]['id']}]}

In [49]:
# Conditional edge
def route_message(state: MessagesState, config: RunnableConfig, store: BaseStore) -> Literal[END, "update_todos", "update_instructions", "update_profile"]:

    """Reflect on the memories and chat history to decide whether to update the memory collection."""
    message = state['messages'][-1]
    if len(message.tool_calls) ==0:
        return END
    else:
        tool_call = message.tool_calls[0]
        if tool_call['args']['update_type'] == "user":
            return "update_profile"
        elif tool_call['args']['update_type'] == "todo":
            return "update_todos"
        elif tool_call['args']['update_type'] == "instructions":
            return "update_instructions"
        else:
            raise ValueError

In [50]:
# define graph
builder = StateGraph(MessagesState)

# add graph nodes
builder.add_node("mAIstro", task_mAIstro)
builder.add_node("update_todos", update_todos)
builder.add_node("update_profile", update_profile)
builder.add_node("update_instructions", update_instructions)

# add edges
builder.add_edge(START, "mAIstro")
builder.add_conditional_edges("mAIstro", route_message)
builder.add_edge("update_todos", "mAIstro")
builder.add_edge("update_profile", "mAIstro")
builder.add_edge("update_instructions", "mAIstro")

# Store for long-term (across-thread) memory
across_thread_memory = InMemoryStore()

# Checkpointer for short-term (within-thread) memory
within_thread_memory = MemorySaver()

# We compile the graph with the checkpointer and store
graph = builder.compile(checkpointer=within_thread_memory, store=across_thread_memory)

In [51]:
# display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [52]:
# We supply a thread ID for short-term (within-thread) memory
# We supply a user ID for long-term (across-thread) memory 
config = {"configurable": {"thread_id": "1", "user_id": "Umer"}}

# User input to create a profile memory
input_messages = [HumanMessage(content="My name is Umer. I live in Karachi with my family. I am single.")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

My name is Umer. I live in Karachi with my family. I am single.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_WfMPCqvYMPXU6EDTXiyA5nmA)
 Call ID: call_WfMPCqvYMPXU6EDTXiyA5nmA
  Args:
    update_type: user


c:\Gen-Agentic-AI-Tutorials\.venv\Lib\site-packages\pydantic\json_schema.py:2448: PydanticJsonSchemaWarning: Default value <class 'list'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


================================= Tool Message =================================

updated profile
================================== Ai Message ==================================

I've updated your profile with your information. If there's anything else you'd like to share or update, just let me know!


In [53]:
# User input for a ToDo
input_messages = [HumanMessage(content="My wife asked me to book swim lessons for the baby.")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

My wife asked me to book swim lessons for the baby.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_Tzcd0xjcxKwoMrx4zbAoD4yJ)
 Call ID: call_Tzcd0xjcxKwoMrx4zbAoD4yJ
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'tasks': ['Book swim lessons for the baby'], 'status': 'not started'}
================================== Ai Message ==================================

I've added the task to book swim lessons for the baby to your ToDo list. If you have any more tasks or updates, feel free to share!


In [54]:
# User input to update instructions for creating ToDos
input_messages = [HumanMessage(content="When creating or updating ToDo items, include specific local businesses / vendors.")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

When creating or updating ToDo items, include specific local businesses / vendors.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_GbdjRBK4oFqQtXfWCl9YTZSA)
 Call ID: call_GbdjRBK4oFqQtXfWCl9YTZSA
  Args:
    update_type: instructions
================================= Tool Message =================================

updated instructions
================================== Ai Message ==================================

I've noted your preference to include specific local businesses or vendors when creating or updating ToDo items. If you have any other preferences or tasks, just let me know!


In [56]:
# Check for updated instructions
user_id = "Umer"

# Search 
for memory in across_thread_memory.search(("instructions", user_id)):
    print(memory.value)

{'memory': 'Based on your feedback, here are the updated instructions for how to update ToDo list items:\n\n1. When creating or updating ToDo items, always include specific local businesses or vendors relevant to the task, if applicable.\n2. Ensure that tasks are clear and actionable, providing any necessary details that may help in completing the task.\n\nIf you have any more preferences or changes, just let me know!'}


In [57]:
# User input for a ToDo
input_messages = [HumanMessage(content="I need to fix the jammed electric Yale lock on the door.")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I need to fix the jammed electric Yale lock on the door.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_IDgQ0wKKDESzWgKto9Fsktsq)
 Call ID: call_IDgQ0wKKDESzWgKto9Fsktsq
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'tasks': ['Fix the jammed electric Yale lock on the door'], 'status': 'not started', 'solutions': ['Contact a local locksmith', 'Check online for DIY repair guides']}

Document 6d2afb8d-404e-401a-a9b5-6fc51cfdacf5 updated:
Plan: Add a new task to fix the jammed electric Yale lock on the door.
Added content: Fix the jammed electric Yale lock on the door
================================== Ai Message ==================================

I've added the task to fix the jammed electric Yale lock on the door to your ToDo list. If you need any 

In [58]:
# Namespace for the memory to save
user_id = "Umer"

# Search 
for memory in across_thread_memory.search(("todo", user_id)):
    print(memory.value)

{'tasks': ['Book swim lessons for the baby', 'Fix the jammed electric Yale lock on the door'], 'time_to_complete': None, 'deadline': None, 'solutions': ['Contact a local locksmith', 'Check online for DIY repair guides'], 'status': 'not started'}
{'tasks': ['Fix the jammed electric Yale lock on the door'], 'time_to_complete': None, 'deadline': None, 'solutions': ['Contact a local locksmith', 'Check online for DIY repair guides'], 'status': 'not started'}


In [59]:
# User input to update an existing ToDo
input_messages = [HumanMessage(content="For the swim lessons, I need to get that done by end of November.")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

For the swim lessons, I need to get that done by end of November.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_jYwN8suDWRAD6IR4rOsuXilT)
 Call ID: call_jYwN8suDWRAD6IR4rOsuXilT
  Args:
    update_type: todo
================================= Tool Message =================================

Document 6d2afb8d-404e-401a-a9b5-6fc51cfdacf5 updated:
Plan: Update the deadline for swim lessons to end of November.
Added content: 2025-11-30T23:59:59

Document cec4f48b-8b05-45a3-844c-f3a639e250c4 updated:
Plan: Update the deadline for swim lessons to end of November.
Added content: 2025-11-30T23:59:59
================================== Ai Message ==================================

I've updated the deadline for the swim lessons to the end of November. If there's anything else you need to add or modify, just let me know!


In [60]:
# User input for a ToDo
input_messages = [HumanMessage(content="Need to call back City Toyota to schedule car service.")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Need to call back City Toyota to schedule car service.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_AumpS2YSeyunaWRV9FvktR2S)
 Call ID: call_AumpS2YSeyunaWRV9FvktR2S
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'tasks': ['Call back City Toyota to schedule car service'], 'status': 'not started'}

New ToDo created:
Content: {'tasks': ['Call back City Toyota to schedule car service'], 'status': 'not started'}
================================== Ai Message ==================================

I've added the task to call back City Toyota to schedule car service to your ToDo list. If you have any more tasks or updates, feel free to share!


In [61]:
# Namespace for the memory to save
user_id = "Umer"

# Search 
for memory in across_thread_memory.search(("todo", user_id)):
    print(memory.value)

{'tasks': ['Book swim lessons for the baby', 'Fix the jammed electric Yale lock on the door'], 'time_to_complete': None, 'deadline': '2025-11-30T23:59:59', 'solutions': ['Contact a local locksmith', 'Check online for DIY repair guides'], 'status': 'not started'}
{'tasks': ['Fix the jammed electric Yale lock on the door'], 'time_to_complete': None, 'deadline': '2025-11-30T23:59:59', 'solutions': ['Contact a local locksmith', 'Check online for DIY repair guides'], 'status': 'not started'}
{'tasks': ['Call back City Toyota to schedule car service'], 'time_to_complete': None, 'deadline': None, 'solutions': [], 'status': 'not started'}
{'tasks': ['Call back City Toyota to schedule car service'], 'time_to_complete': None, 'deadline': None, 'solutions': [], 'status': 'not started'}


In [62]:
# We supply a thread ID for short-term (within-thread) memory
# We supply a user ID for long-term (across-thread) memory 
config = {"configurable": {"thread_id": "2", "user_id": "Umer"}}

# Chat with the chatbot
input_messages = [HumanMessage(content="I have 30 minutes, what tasks can I get done?")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I have 30 minutes, what tasks can I get done?
================================== Ai Message ==================================

You have a couple of tasks on your ToDo list that you might be able to tackle in 30 minutes:

1. **Call back City Toyota to schedule car service** - This task doesn't have a specific time requirement, so it could likely be completed within your available time.

Would you like to focus on this task, or do you want to prioritize something else?


In [63]:
# Chat with the chatbot
input_messages = [HumanMessage(content="Yes, give me some options to call for swim lessons.")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Yes, give me some options to call for swim lessons.
================================== Ai Message ==================================

Here are a few options for swim lessons in Karachi that you can consider:

1. **Swim Karachi** - They offer various swimming classes for children and adults. You can check their website or call them for more details.

2. **Aqua Fitness** - Known for their structured swim programs for kids. They might have flexible timings that suit your schedule.

3. **The Karachi Swim School** - They provide lessons for all ages and skill levels. It’s worth reaching out to see what they offer for babies.

4. **Local Community Pools** - Many community pools offer swim lessons for children. You can check with your local community center for options.

Would you like to proceed with contacting one of these options, or do you need more information?


We can see that Trustcall performs patching of the existing memory:

https://smith.langchain.com/public/4ad3a8af-3b1e-493d-b163-3111aa3d575a/r

Now we can create a new thread.

This creates a new session. 

Profile, ToDos, and Instructions saved to long-term memory are accessed. 

Trace: 

https://smith.langchain.com/public/84768705-be91-43e4-8a6f-f9d3cee93782/r

## Studio

![Screenshot 2024-11-04 at 1.00.19 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/6732cfb05d9709862eba4e6c_Screenshot%202024-11-11%20at%207.46.40%E2%80%AFPM.png)